### Student Information
Name: 陳羿先

Student ID: 108062506

GitHub ID: s38565

Kaggle name: sh1llyixian

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM19-Lab2-Master Repo](https://github.com/EvaArevalo/DM19-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/179d01d4dd984fc5ac45a894822479dd) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the score (ie. 20% of 30% )

    - **Top 41% - 100%**: Get (101-x)% of the score, where x is your ranking in the leaderboard (ie. (101-x)% of 30% )   
    Submit your last submission __BEFORE the deadline (Nov. 23rd 11:59 pm, Saturday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/EvaArevalo/DM19-Lab1-Master/blob/master/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb), but make sure to fork the [DM19-Lab2-Homework](https://github.com/EvaArevalo/DM19-Lab2-Homework) repository this time! Also please __DON´T UPLOAD HUGE DOCUMENTS__, please use Git ignore for that.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)__. 

In [1]:
# import related library
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import re, csv
from nltk.corpus import stopwords
import nltk
import string

In this homework, I first try deep learning NN model as lab showed, but I finally found that it's not a good model due to my bad prediction result.<br>
So the first part of the code is to show my trying process.<br>
And the second part is the LSTM model I used in compitition.<br>

### 1. Loading related raw data

In this part, I load all data in and merge them into single dataframe format and finally seperate train/test.

In [3]:
# load csv and restructure dataframe (rename)
iden_df = pd.read_csv("data_identification.csv", sep="\t")
iden_df['tweet_id'] = iden_df['tweet_id,identification'].str.split(',', expand=True)[0]
iden_df['identification'] = iden_df['tweet_id,identification'].str.split(',', expand=True)[1]
iden_df = iden_df.drop('tweet_id,identification', axis=1)

In [4]:
iden_df.head() # show

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [5]:
# see shape of dataframe
iden_df.shape

(1867535, 2)

In [6]:
# load csv and restructure dataframe (rename)
emotion_df = pd.read_csv("emotion.csv", sep="\t")
emotion_df['tweet_id'] = emotion_df['tweet_id,emotion'].str.split(',', expand=True)[0]
emotion_df['emotion'] = emotion_df['tweet_id,emotion'].str.split(',', expand=True)[1]
emotion_df = emotion_df.drop('tweet_id,emotion', axis=1)

In [7]:
emotion_df.head() # show


,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [8]:
# see shape of dataframe
emotion_df.shape

(1455563, 2)

In [42]:
# load raw data of twitter
datas = open('tweets_DM.json').read()
tweet_DM = pd.read_json(datas, lines = True)
tweet_DM.head()

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [43]:
# get some useful information from all data
train_df = json_normalize(tweet_DM['_source'])
train_df['tweet_id'] = train_df['tweet.tweet_id']
train_df = train_df.drop('tweet.tweet_id', axis=1)
train_df['text'] = train_df['tweet.text']
train_df = train_df.drop('tweet.text', axis=1)
train_df['hashtags'] = train_df['tweet.hashtags']
train_df = train_df.drop('tweet.hashtags', axis=1)

In [44]:
train_df.head() # show

,tweet_id,text,hashtags
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat]
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]"
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse]
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[]
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[]


In [45]:
# see shape of dataframe
train_df.shape 

(1867535, 3)

In [46]:
# check if duplicated
emotion_df.duplicated(keep='first')
iden_df.duplicated(keep='first')

0          False
1          False
2          False
3          False
4          False
           ...  
1867530    False
1867531    False
1867532    False
1867533    False
1867534    False
Length: 1867535, dtype: bool

In [47]:
# merge data sets
train_df_merge = pd.merge(train_df, emotion_df, how='outer', on=['tweet_id'])
train_df_merge.head(10)

,tweet_id,text,hashtags,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],NaN
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[],NaN
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation
7,0x368e95,Love knows no gender. 😢😭 <LH>,[],joy
8,0x249c0c,@DStvNgCare @DStvNg More highlights are being ...,[LeagueCup],sadness
9,0x218443,When do you have enough ? When are you satisfi...,"[materialism, money, possessions]",NaN


In [48]:
# merge data sets
train_df_merge = pd.merge(train_df_merge, iden_df, how='outer', on=['tweet_id'])
train_df_merge.head(10)

,tweet_id,text,hashtags,emotion,identification
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],NaN,test
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[],NaN,test
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train
7,0x368e95,Love knows no gender. 😢😭 <LH>,[],joy,train
8,0x249c0c,@DStvNgCare @DStvNg More highlights are being ...,[LeagueCup],sadness,train
9,0x218443,When do you have enough ? When are you satisfi...,"[materialism, money, possessions]",NaN,test


In [49]:
# seperate training and testing data
trains, tests = [x for _, x in train_df_merge.groupby(train_df_merge['identification'] == 'test')]

In [55]:
# see shape of dataframe
trains.shape

(1455563, 5)

In [80]:
# see shape of dataframe
tests.shape

(411972, 5)

In [102]:
tests.head() #show

,tweet_id,text,hashtags,emotion,identification
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],NaN,test
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[],NaN,test
9,0x218443,When do you have enough ? When are you satisfi...,"[materialism, money, possessions]",NaN,test
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #...","[GodsPlan, GodsWork]",NaN,test
33,0x26289a,"In these tough times, who do YOU turn to as yo...",[],NaN,test


In [50]:
## save to pickle file
trains.to_pickle("train_df.pkl") 
tests.to_pickle("test_df.pkl")

In [2]:
## load a pickle file
trains = pd.read_pickle("train_df.pkl")
tests = pd.read_pickle("test_df.pkl")

In [3]:
#group to find distribution
trains.groupby(['emotion']).count()['text']

emotion
anger            39867
anticipation    248935
disgust         139101
fear             63999
joy             516017
sadness         193437
surprise         48729
trust           205478
Name: text, dtype: int64

### 2. Preprocessing

Related Work by: https://medium.com/@SeoJaeDuk/basic-data-cleaning-engineering-session-twitter-sentiment-data-b9376a91109b

### (1). remove stopwords from nltk library (cleaning)

In the first step of my preprocessing, I use nltk english stopwords library to clean my data.<br>
This step can remove all not related vocabulary(not such meaningful for us) such as I,we, then......

In [3]:
# remove stop words
stop = stopwords.words('english')
trains['stop_words'] = trains['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [118]:
trains.head() # show

,tweet_id,text,hashtags,emotion,identification,stop_words
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"People post ""add #Snapchat"" must dehydrated. C..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"@brianklaas As see, Trump dangerous #freepress..."
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,Now ISSA stalking Tasha 😂😂😂 <LH>
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train,@RISKshow @TheKevinAllison Thx BEST TIME tonig...
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train,Still waiting supplies Liscus. <LH>


### (2). Replace abbreviations and some spell correction

In twitter, there are sometimes we'll talk colloquially.<br>
So, we need to use some translator to change these kinds of abbreviation back.<br>
And this can be done by slang.txt.

In [4]:
# function to translate abbreviations
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
        j = j + 1
    # Replacing commas with spaces for final output.
    return (' '.join(user_string))



In [5]:
# add columns for new step of preprocessing
trains['sentimentText'] = trains['stop_words'].apply(lambda x: translator(x))

In [130]:
trains.head() # show

,tweet_id,text,hashtags,emotion,identification,stop_words,sentimentText
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"People post ""add #Snapchat"" must dehydrated. C...","People post ""add #Snapchat"" must dehydrated. C..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"@brianklaas As see, Trump dangerous #freepress...","@brianklaas As see, Trump dangerous #freepress..."
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,Now ISSA stalking Tasha 😂😂😂 <LH>,Now ISSA stalking Tasha 😂😂😂 <LH>
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train,@RISKshow @TheKevinAllison Thx BEST TIME tonig...,@RISKshow @TheKevinAllison Thank You BEST TIME...
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train,Still waiting supplies Liscus. <LH>,Still waiting supplies Liscus. <LH>


### (3). Stemming

As we known, stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form.<br>
And this step can prevent us from treating different part-of-speech  as different features and misidentify.

In [6]:
# step 3 Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
trains['stemming'] = trains['sentimentText'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))

In [132]:
trains.head() #show

,tweet_id,text,hashtags,emotion,identification,stop_words,sentimentText,stemming
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"People post ""add #Snapchat"" must dehydrated. C...","People post ""add #Snapchat"" must dehydrated. C...","peopl post ""add #snapchat"" must dehydrated. cu..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"@brianklaas As see, Trump dangerous #freepress...","@brianklaas As see, Trump dangerous #freepress...","@brianklaa As see, trump danger #freepress aro..."
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,Now ISSA stalking Tasha 😂😂😂 <LH>,Now ISSA stalking Tasha 😂😂😂 <LH>,now issa stalk tasha 😂😂😂 <lh>
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train,@RISKshow @TheKevinAllison Thx BEST TIME tonig...,@RISKshow @TheKevinAllison Thank You BEST TIME...,@riskshow @thekevinallison thank you best time...
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train,Still waiting supplies Liscus. <LH>,Still waiting supplies Liscus. <LH>,still wait suppli liscus. <lh>


### (4). Lemmatize

Lemmatization is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word’s lemma, or dictionary form.<br>
And this process is very similar to stemming.<br>
However one difference we can note is the fact that the word already have not changed to alreadi. <br>
So I finally just keep this process.

In [7]:
# step 4 Lemmatize
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
trains['lemmazation'] = trains['stemming'].apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split()]))

In [134]:
trains.head()

,tweet_id,text,hashtags,emotion,identification,stop_words,sentimentText,stemming,lemmazation
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"People post ""add #Snapchat"" must dehydrated. C...","People post ""add #Snapchat"" must dehydrated. C...","peopl post ""add #snapchat"" must dehydrated. cu...","peopl post ""add #snapchat"" must dehydrated. cu..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"@brianklaas As see, Trump dangerous #freepress...","@brianklaas As see, Trump dangerous #freepress...","@brianklaa As see, trump danger #freepress aro...","@brianklaa As see, trump danger #freepress aro..."
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,Now ISSA stalking Tasha 😂😂😂 <LH>,Now ISSA stalking Tasha 😂😂😂 <LH>,now issa stalk tasha 😂😂😂 <lh>,now issa stalk tasha 😂😂😂 <lh>
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train,@RISKshow @TheKevinAllison Thx BEST TIME tonig...,@RISKshow @TheKevinAllison Thank You BEST TIME...,@riskshow @thekevinallison thank you best time...,@riskshow @thekevinallison thank you best time...
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train,Still waiting supplies Liscus. <LH>,Still waiting supplies Liscus. <LH>,still wait suppli liscus. <lh>,still wait suppli liscus. <lh>


### (5).  Remove punctuation

After above preprocessing process, I found that punctuation will be some noise information in training.<br>
In addition, Twitter contains many #hashtag in text data. <br>
If we just remove hashtag word, I think this will lose many useful information. <br>
So I use string libraby to remove all kinds of punctuation but keep hashtag word. <br>
ex: #sad -> sad <br>

In [9]:
# remove punctuation
translator = str.maketrans('','',string.punctuation)
trains['cleaned'] = trains['lemmazation'].str.replace('[{}]'.format(string.punctuation), '')

In [10]:
trains.head()

,tweet_id,text,hashtags,emotion,identification,stop_words,sentimentText,stemming,lemmazation,text_tokenized,cleaned
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"People post ""add #Snapchat"" must dehydrated. C...","People post ""add #Snapchat"" must dehydrated. C...","peopl post ""add #snapchat"" must dehydrated. cu...","peopl post ""add #snapchat"" must dehydrated. cu...","[peopl, post, add, snapchat, must, dehydrated,...",peopl post add snapchat must dehydrated cuz ma...
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"@brianklaas As see, Trump dangerous #freepress...","@brianklaas As see, Trump dangerous #freepress...","@brianklaa As see, trump danger #freepress aro...","@brianklaa As see, trump danger #freepress aro...","[brianklaa, As, see, trump, danger, freepress,...",brianklaa As see trump danger freepress around...
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,Now ISSA stalking Tasha 😂😂😂 <LH>,Now ISSA stalking Tasha 😂😂😂 <LH>,now issa stalk tasha 😂😂😂 <lh>,now issa stalk tasha 😂😂😂 <lh>,"[now, issa, stalk, tasha, 😂😂😂, lh]",now issa stalk tasha 😂😂😂 lh
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train,@RISKshow @TheKevinAllison Thx BEST TIME tonig...,@RISKshow @TheKevinAllison Thank You BEST TIME...,@riskshow @thekevinallison thank you best time...,@riskshow @thekevinallison thank you best time...,"[riskshow, thekevinallison, thank, you, best, ...",riskshow thekevinallison thank you best time t...
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train,Still waiting supplies Liscus. <LH>,Still waiting supplies Liscus. <LH>,still wait suppli liscus. <lh>,still wait suppli liscus. <lh>,"[still, wait, suppli, liscus, lh]",still wait suppli liscus lh


### (6). Related preprocessing method I tried

I tried many different preprocessing method, and the below are what I tried but didn't use finally.

 text_tokenize

In [8]:
# text_tokenized from texts to lists of words
import gensim

## ignore warnings
import warnings
warnings.filterwarnings('ignore')
translator = str.maketrans('','',string.punctuation)

trains['text_tokenized'] = trains['lemmazation'].apply(lambda x: nltk.word_tokenize(x.translate(translator)))
trains[['tweet_id', 'lemmazation', 'text_tokenized']].head()


,tweet_id,lemmazation,text_tokenized
0,0x376b20,"peopl post ""add #snapchat"" must dehydrated. cu...","[peopl, post, add, snapchat, must, dehydrated,..."
1,0x2d5350,"@brianklaa As see, trump danger #freepress aro...","[brianklaa, As, see, trump, danger, freepress,..."
3,0x1cd5b0,now issa stalk tasha 😂😂😂 <lh>,"[now, issa, stalk, tasha, 😂😂😂, lh]"
5,0x1d755c,@riskshow @thekevinallison thank you best time...,"[riskshow, thekevinallison, thank, you, best, ..."
6,0x2c91a8,still wait suppli liscus. <lh>,"[still, wait, suppli, liscus, lh]"


change emoji to words based on emoji library

In [4]:
import emoji
trains['c1'] = trains['cleaned'].apply(lambda x: (emoji.demojize(x)))
trains[['tweet_id', 'lemmazation', 'text_tokenized', 'c1']].head()

,tweet_id,lemmazation,text_tokenized,c1
0,0x376b20,"peopl post ""add #snapchat"" must dehydrated. cu...","[peopl, post, add, snapchat, must, dehydrated,...",peopl post add snapchat must dehydrated cuz ma...
1,0x2d5350,"@brianklaa As see, trump danger #freepress aro...","[brianklaa, As, see, trump, danger, freepress,...",brianklaa As see trump danger freepress around...
3,0x1cd5b0,now issa stalk tasha 😂😂😂 <lh>,"[now, issa, stalk, tasha, 😂😂😂, lh]",now issa stalk tasha :face_with_tears_of_joy::...
5,0x1d755c,@riskshow @thekevinallison thank you best time...,"[riskshow, thekevinallison, thank, you, best, ...",riskshow thekevinallison thank you best time t...
6,0x2c91a8,still wait suppli liscus. <lh>,"[still, wait, suppli, liscus, lh]",still wait suppli liscus lh


In [5]:
translator = str.maketrans('','',string.punctuation)
trains['c2'] = trains['c1'].str.replace('[{}]'.format(string.punctuation), '')

In [6]:
trains[['tweet_id', 'lemmazation', 'text_tokenized', 'c2']].head()

,tweet_id,lemmazation,text_tokenized,c2
0,0x376b20,"peopl post ""add #snapchat"" must dehydrated. cu...","[peopl, post, add, snapchat, must, dehydrated,...",peopl post add snapchat must dehydrated cuz ma...
1,0x2d5350,"@brianklaa As see, trump danger #freepress aro...","[brianklaa, As, see, trump, danger, freepress,...",brianklaa As see trump danger freepress around...
3,0x1cd5b0,now issa stalk tasha 😂😂😂 <lh>,"[now, issa, stalk, tasha, 😂😂😂, lh]",now issa stalk tasha facewithtearsofjoyfacewit...
5,0x1d755c,@riskshow @thekevinallison thank you best time...,"[riskshow, thekevinallison, thank, you, best, ...",riskshow thekevinallison thank you best time t...
6,0x2c91a8,still wait suppli liscus. <lh>,"[still, wait, suppli, liscus, lh]",still wait suppli liscus lh


### (7). Our new dataframe after preprocessing

In [117]:
trains.head()

,tweet_id,text,hashtags,emotion,identification,stop_words,sentimentText,stemming,lemmazation,text_tokenized,cleaned,c1,c2
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"People post ""add #Snapchat"" must dehydrated. C...","People post ""add #Snapchat"" must dehydrated. C...","peopl post ""add #snapchat"" must dehydrated. cu...","peopl post ""add #snapchat"" must dehydrated. cu...","[peopl, post, add, snapchat, must, dehydrated,...",peopl post add snapchat must dehydrated cuz ma...,peopl post add snapchat must dehydrated cuz ma...,peopl post add snapchat must dehydrated cuz ma...
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"@brianklaas As see, Trump dangerous #freepress...","@brianklaas As see, Trump dangerous #freepress...","@brianklaa As see, trump danger #freepress aro...","@brianklaa As see, trump danger #freepress aro...","[brianklaa, As, see, trump, danger, freepress,...",brianklaa As see trump danger freepress around...,brianklaa As see trump danger freepress around...,brianklaa As see trump danger freepress around...
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,Now ISSA stalking Tasha 😂😂😂 <LH>,Now ISSA stalking Tasha 😂😂😂 <LH>,now issa stalk tasha 😂😂😂 <lh>,now issa stalk tasha 😂😂😂 <lh>,"[now, issa, stalk, tasha, 😂😂😂, lh]",now issa stalk tasha 😂😂😂 lh,now issa stalk tasha :face_with_tears_of_joy::...,now issa stalk tasha facewithtearsofjoyfacewit...
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy,train,@RISKshow @TheKevinAllison Thx BEST TIME tonig...,@RISKshow @TheKevinAllison Thank You BEST TIME...,@riskshow @thekevinallison thank you best time...,@riskshow @thekevinallison thank you best time...,"[riskshow, thekevinallison, thank, you, best, ...",riskshow thekevinallison thank you best time t...,riskshow thekevinallison thank you best time t...,riskshow thekevinallison thank you best time t...
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],anticipation,train,Still waiting supplies Liscus. <LH>,Still waiting supplies Liscus. <LH>,still wait suppli liscus. <lh>,still wait suppli liscus. <lh>,"[still, wait, suppli, liscus, lh]",still wait suppli liscus lh,still wait suppli liscus lh,still wait suppli liscus lh


And we can save them as pickle file!

In [12]:
## save to pickle file
trains.to_pickle("train_cleaned.pkl") 
tests.to_pickle("test_cleaned.pkl")

In [2]:
## load a pickle file
trains = pd.read_pickle("train_cleaned.pkl")
tests = pd.read_pickle("test_cleaned.pkl")

 ### 2-1. feature engineering steps - TFIDF

In feature engineering steps, I have tried TF-IDF and BOW to select my feature.

In [118]:
# seperate training and validation data

sep = int(trains.shape[0] * 0.8) # 20% for validation

BOW_train = trains[:sep]
BOW_train_label = trains.emotion[:sep]
BOW_test = trains[sep:]
BOW_test_label = trains.emotion[sep:]

print(BOW_train.shape)
print(BOW_train_label.shape)
print(BOW_test.shape)
print(BOW_test_label.shape)

(1164450, 13)
(1164450,)
(291113, 13)
(291113,)


In [26]:

# TF-IDF feature selecting

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1500, use_idf=True)
tfidf_vectorizer.fit(BOW_train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1500, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [17]:
BOW_train = BOW.transform(trains['cleaned'])

In [11]:
tfidf_vectorizer.get_feature_names()[-100:] # show features

['yield',
 'yikes',
 'ynwa',
 'yo',
 'yoga',
 'yogi',
 'yolo',
 'york',
 'yorkshir',
 'you',
 'youcandoit',
 'youd',
 'youll',
 'young',
 'younger',
 'youngest',
 'your',
 'youredheart',
 'yourock',
 'yours',
 'yourself',
 'yourselves',
 'youth',
 'youthinkingface',
 'youtub',
 'youtube',
 'youu',
 'youv',
 'yr',
 'yrs',
 'yt',
 'yu',
 'yuck',
 'yum',
 'yummi',
 'yup',
 'yvonneorji',
 'yyc',
 'z',
 'z100newyork',
 'z1043',
 'zach',
 'zakbagan',
 'zakhmi',
 'zanyface',
 'zapti',
 'zaynmalik',
 'zbczone',
 'zealand',
 'zeke',
 'zen',
 'zero',
 'zimbabw',
 'zion',
 'zip',
 'zombi',
 'zone',
 'zoo',
 'zzz',
 '\x92',
 '¡',
 '£',
 '«',
 '¯',
 '°',
 '´',
 '·',
 '»',
 '¿',
 '×',
 'ा',
 'ी',
 '्',
 'ღ',
 '\u200b',
 '–',
 '—',
 '―',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '…',
 '€',
 '━',
 '┃',
 '┈',
 '█',
 '░',
 '★',
 '☆',
 '♡',
 '♪',
 '♫',
 '⚘',
 '�',
 '🕆',
 '🕇',
 '🖒']

In [33]:
sep = int(trains.shape[0] * 0.8) # 20% for validation


BOW_train_X = BOW_train[:sep]
BOW_train_y = trains.emotion[:sep]

BOW_test_X = BOW_train[sep:]
BOW_test_y = trains.emotion[sep:]

## take a look at data dimension is a good habbit  :)
print('BOW_train_X.shape: ', BOW_train_X.shape)
print('BOW_train_y.shape: ', BOW_train_y.shape)
print('BOW_test_X.shape: ', BOW_test_X.shape)
print('BOW_test_y.shape: ', BOW_test_y.shape)

BOW_train_X.shape:  (1164450, 11)
BOW_train_y.shape:  (1164450,)
BOW_test_X.shape:  (0, 11)
BOW_test_y.shape:  (291113,)


In [34]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(BOW_train_y)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:7]:\n', BOW_train_y[0:7])
print('\ny_train.shape: ',BOW_train_y.shape)
print('y_test.shape: ', BOW_test_y.shape)


check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:7]:
 0    anticipation
1         sadness
3            fear
5             joy
6    anticipation
7             joy
8         sadness
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


In [35]:
import keras
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

BOW_train_y = label_encode(label_encoder, BOW_train_y)
BOW_test_y = label_encode(label_encoder, BOW_test_y)

print('\n\n## After convert')
print('y_train[0:7]:\n', BOW_train_y[0:7])
print('\ny_train.shape: ', BOW_train_y.shape)
print('y_test.shape: ', BOW_test_y.shape)




## After convert
y_train[0:7]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)


In [36]:
# I/O check
input_shape = BOW_train_X.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  11
output_shape:  8


 ### 3-1. Deep learning Model - TFIDF

In [22]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import regularizers

# input layer
model_input = Input(shape=(input_shape, ))  # 3000
X = model_input

# 1st hidden layer

X_W1 = Dense(units=512, kernel_regularizer = regularizers.l2(0.001), activity_regularizer=regularizers.l1(0.01))(X)  # 64
H1 = BatchNormalization()(X_W1)
H1 = ReLU()(H1)
H1_Drop = Dropout(0.4)(H1)

# 2nd hidden layer
X_W2 = Dense(units=256, kernel_regularizer = regularizers.l2(0.001), activity_regularizer=regularizers.l1(0.01))(H1_Drop)  # 64
H2 = BatchNormalization()(X_W2)
H2 = ReLU()(H2)
H2_Drop = Dropout(0.4)(H2)


# 3rd hidden layer
X_W3 = Dense(units=64, kernel_regularizer = regularizers.l2(0.001), activity_regularizer=regularizers.l1(0.01))(H2_Drop)  # 64
H3 = BatchNormalization()(X_W3)
H3 = ReLU()(H3)
H3_Drop = Dropout(0.4)(H3)

# 4th hidden layer
X_W4 = Dense(units=64, kernel_regularizer = regularizers.l2(0.001), activity_regularizer=regularizers.l1(0.01))(H3_Drop)  # 64
H4 = BatchNormalization()(X_W4)
H4 = ReLU()(H4)
H4_Drop = Dropout(0.4)(H4)

# 5th hidden layer
X_W5 = Dense(units=32, kernel_regularizer = regularizers.l2(0.001), activity_regularizer=regularizers.l1(0.01))(H4_Drop)  # 64
H5 = BatchNormalization()(X_W5)
H5 = ReLU()(H5)
H5_Drop = Dropout(0.4)(H5)

# output layer
X_H6 = Dense(units=output_shape)(H5_Drop)  # 4
H6 = Softmax()(X_H6)

model_output = H6

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1536512   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
____________________________________________

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log_BOW.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(BOW_train_X, BOW_train_y, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (BOW_test_X, BOW_test_y))
print('training finish')

 ### 2-2. feature engineering steps - BOW

In [7]:
sep = int(trains.shape[0] * 0.8) # 20% for validation

BOW_train = trains[:sep]
BOW_train_label = trains.emotion[:sep]
BOW_test = trains[sep:]
BOW_test_label = trains.emotion[sep:]

print(BOW_train.shape)
print(BOW_train_label.shape)
print(BOW_test.shape)
print(BOW_test_label.shape)

(1164450, 13)
(1164450,)
(291113, 13)
(291113,)


In [16]:
# Bag of word feature selecting

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer()

# build analyzers (bag-of-words)
BOW = CountVectorizer(analyzer='word', max_features=3000, max_df=0.8, tokenizer=tknzr.tokenize)

# apply analyzer to training data
BOW.fit(trains['cleaned'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=3000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f1a69508940>>,
        vocabulary=None)

In [24]:
import keras

# standardize name (X, y) 
X_train = tfidf_vectorizer.transform(BOW_train['c2'])
y_train = BOW_train_label

X_test = tfidf_vectorizer.transform(BOW_test['c2'])
y_test = BOW_test_label

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1164450, 5000)
y_train.shape:  (1164450,)
X_test.shape:  (291113, 5000)
y_test.shape:  (291113,)


In [25]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)



check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:7]:
 0    anticipation
1         sadness
3            fear
5             joy
6    anticipation
7             joy
8         sadness
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


In [26]:
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)




## After convert
y_train[0:7]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)


In [27]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  5000
output_shape:  8


 ### 3-2. Deep learning model - BOW

In [29]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from sklearn.linear_model import SGDClassifier

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer



X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# H2_W3 = Dense(units=64)(H2)  # 64
# H3 = ReLU()(H2_W3)

# H3_W4 = Dense(units=64)(H3)  # 64
# H4 = ReLU()(H3_W4)

# H4_W5 = Dense(units=64)(H4)  # 64
# H5 = ReLU()(H4_W5)

# H5_W6 = Dense(units=64)(H5)  # 64
# H6 = ReLU()(H5_W6)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 8
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                320064    
_________________________________________________________________
re_lu_7 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
re_lu_8 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 520       
_________________________________________________________________
softmax_4 (Softmax)          (None, 8)                 0   

In [30]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log_1128.csv')

# training setting
epochs = 10
batch_size = 300

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')

Train on 1164450 samples, validate on 291113 samples
Epoch 1/10
 142200/1164450 [==>...........................] - ETA: 20s - loss: 1.5393 - accuracy: 0.444

limit_output extension: Maximum message size of 10000 exceeded with 10074 characters

### 4. Prediction Step

In [34]:
## predict
pred_result = model.predict(X_test, batch_size=150)

In [35]:
# show result of testing accuracy
pred_result = label_decode(label_encoder, pred_result)
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

testing accuracy: 0.52


In [37]:
testing = TfidfVectorizer(max_features=5000, use_idf=True, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', tokenizer=tknzr.tokenize)
testing.fit(tests['text'])
testing = testing.transform(tests['text'])
results = model.predict(testing, batch_size=150)

In [38]:
results = label_decode(label_encoder, results)

In [39]:
results[:5]

array(['disgust', 'anticipation', 'sadness', 'disgust', 'joy'],
      dtype=object)

In [40]:
tests['tweet_id'].head()

2     0x28b412
4     0x2de201
9     0x218443
30    0x2939d5
33    0x26289a
Name: tweet_id, dtype: object

In [41]:
# store prediction to csv file
pred = tests

In [42]:
pred['emotion'] = results

In [43]:
pred = pred.drop('hashtags', axis=1)
pred = pred.drop('text', axis=1)
pred = pred.drop('identification', axis=1)

In [44]:
pred.rename(columns={'tweet_id':'id'}, inplace=True)

In [45]:
pred.head()

,id,emotion
2,0x28b412,disgust
4,0x2de201,anticipation
9,0x218443,sadness
30,0x2939d5,disgust
33,0x26289a,joy


In [46]:
pred.to_csv('prediction.csv', index=False)

## Kaggle compition model - LSTM

In [ ]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import re, csv
from nltk.corpus import stopwords
import nltk
import string
from nltk import word_tokenize
from gensim.models import KeyedVectors   
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer


import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, ReLU, Softmax, LSTM, Dropout, Embedding, Bidirectional
from keras.callbacks import CSVLogger



from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.initializers import Constant


### 1. Preprocessing step (the same)

In [ ]:
# load cleaned data
trains = pd.read_pickle("train_cleaned.pkl")
tests = pd.read_pickle("test_cleaned.pkl")

In [ ]:
trains.head() # show

### 2. feature engineering steps

This time, I use pretrained Word2Vec model(English Common Crawl Corpus) to be my feature choosing.<br>

In [ ]:
# use Word2vec Model for feature
word2vec_model = KeyedVectors.load_word2vec_format("model_ENC3.bin", binary=True)

And I use Tokenizer() to filter all punctuations, in order to make tf-idf more better.

In [ ]:
# tokenizer initialize
from keras.preprocessing.text import Tokenizer
tknr = Tokenizer(num_words=None, 
               filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', 
               lower=True, 
               split=' ', 
               char_level=False, 
               oov_token=None)

Using TFIDF to select features

In [ ]:

MAX_WORD = 80000 # max feature selecting

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    max_features=MAX_WORD, 
    use_idf=True, 
    token_pattern='(?u)\\b[^\\d\\W][^\\d\\W]+\\b', 
    )
tfidf_vectorizer.fit(trains['cleaned'])

Use tf-idf results to construct a token_dictionay (contain all MAX_WORD features)<br>
key: feature_name<br>
value: index of feature

In [ ]:
token_dict = {}
for idx, token in enumerate(tfidf_vectorizer.get_feature_names()):
    token_dict[token] = idx

Then, we have to encode all features to (300,1) vector <br>
because our model only accept fixed size of input.

In [ ]:
EMBEDDING_DIM = 300

# return a matrix which all elements are (300,1) vectors
def word2vec_matrix():
    matrix_emb = np.zeros((len(token_dict)+1, EMBEDDING_DIM)) # replace MAX_WORD to len(token_dict)
    for word, idx in token_dict.items():
        try:
            vector =  word2vec_model.wv[word.lower()] # remember to use word.lower() to search for word2vec
        except:
            vector = np.zeros(300,) # if didn't find word vector, just keep vector to be all zero
        if idx < MAX_WORD:
            matrix_emb[idx] = np.array(vector)  # define
    matrix_emb[MAX_WORD] = np.zeros(300,)     # end of vector 
    return matrix_emb

Define function to deal with test_data preprocessing

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from nltk.stem import PorterStemmer
ps = PorterStemmer()

MAX_SEQUENCE_LENGTH = 50

# preprocessing step
def translator1(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
        j = j + 1
    # Replacing commas with spaces for final output.
    return (' '.join(user_string))


# do all my preprocessing steps again
def cleaning(texts):
    
    
    stop = stopwords.words('english')
    texts = texts.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    texts = texts.apply(lambda x: translator1(x))
    texts = texts.apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
    texts = texts.apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split()]))
    translator = str.maketrans('','',string.punctuation)
    texts = texts.str.replace('[{}]'.format(string.punctuation), '')
    return texts



Deal with input Twitter texts:<br>
Use a for loop to travel all rows of our cleaning dataframe and<br>
1. Truncate it(if the word can be found in feature map then set it to be idx of fhat feature, or set it to be MAX_WORD)
2. Add padding if length of that rows of texts < MAX_SEQUENCE_LENGTH  <br>
(make sure each texts after transform have same length)
3. Output a nparay to be our model input

In [ ]:
def text_to_sequence(texts):
    texts = cleaning(texts)
    
    tokenized_texts = []
    for string in texts:
        tokenized_text = []
        for idx, word in enumerate(string.split(' ')):
            
            # truncate
            if idx >= MAX_SEQUENCE_LENGTH: break 
             
            try:
                token = token_dict[word.lower()]
            except:
                token = MAX_WORD
            tokenized_text.append(token)
            
        # padding 
        if len(tokenized_text) < MAX_SEQUENCE_LENGTH: 
            tokenized_text.extend([MAX_WORD]*(MAX_SEQUENCE_LENGTH-len(tokenized_text)))
                
        tokenized_texts.append(tokenized_text)
    return np.array(tokenized_texts)

encode emotion

In [ ]:
# a dictionary to record different emotions types
emotion_array = ['joy', 'sadness', 'trust', 'anticipation', 'fear', 'anger', 'disgust', 'surprise']

Encode emotion from text to idx<br>
to_categorical -> change to matrix

In [ ]:
# encode emotion from "text" to "number"(construct a dict to record)
def emotion_to_sequence(emotions):
    tokenized_emotions = []
    for string in emotions:
        token = emotion_array.index(string)
        tokenized_emotions.append(token)
    return to_categorical(tokenized_emotions)

Use functions above to deal with cleaned text datas

In [ ]:
train_sequence = text_to_sequence(trains['cleaned']) # throw to model
train_label = emotion_to_sequence(trains['emotion']) # answer for training data
my_testing = text_to_sequence(trains['text']) # data for calculating f1-score
text_sequence = text_to_sequence(tests['text']) # throw to model for testing

### 3. Model - LSTM

In [ ]:
# 256,512,512,batch512 -> 不錯不過要加大epoch -> score: 0.39


from sklearn.metrics import accuracy_score
from datetime import datetime 

# layer units information
LSTM1_DIM = 256
LSTM2_DIM = 128

DENSE1_DIM = 512
DENSE2_DIM = 512
DENSE3_DIM = 512

CATEGORY_NUM = 8 # we have 8 different emotions 
BATCH_SIZE = 512
VALIDATION_SPLIT = 0.2 # 80% for training, 20% for validation
EPOCHS = 5

# logger to save information
csv_logger = CSVLogger('training_log.csv')
                    

def model_setting(texts, labels, matrix_emb="uniform"):

    import numpy as np
    import pandas as pd
    DROPOUT = 0.3 # prevent overfitting
    
    if matrix_emb != "uniform": matrix_emb = Constant(matrix_emb)
        
    # split data to train(0.8) & validation(0.2)
    validation_size = int(texts.shape[0]*VALIDATION_SPLIT)
    texts_train = texts[:-validation_size]
    texts_val = texts[-validation_size:]
    labels_train = labels[:-validation_size]
    labels_val = labels[-validation_size:]
    
    # model build
    model = Sequential()
    # embedding layer -> deal with Word2vec input
    model.add(Embedding(MAX_WORD+1, EMBEDDING_DIM, embeddings_initializer=matrix_emb, input_length=MAX_SEQUENCE_LENGTH, trainable=True)) # with embedding matrix
    # LSTM layer (add dropout to prevent from overfitting)
    model.add(LSTM(LSTM1_DIM, dropout=DROPOUT, recurrent_dropout=DROPOUT))
    # dense layer1
    model.add(Dense(DENSE1_DIM, kernel_initializer='normal', activation='relu'))
    # dropout layer
    model.add(Dropout(DROPOUT))
    # dense layer2
    model.add(Dense(DENSE2_DIM, kernel_initializer='normal', activation='relu'))
    # dense layer3
    model.add(Dense(DENSE3_DIM, kernel_initializer='normal', activation='relu'))
    
    # output layer
    model.add(Dense(CATEGORY_NUM, kernel_initializer='normal', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
    model.summary()

    
    history_callback = model.fit(
        texts_train, 
        labels_train, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        callbacks=[csv_logger],
        verbose=1, 
        shuffle=True, 
        validation_data=(texts_val, labels_val))

    return history_callback, model

In [ ]:
history_callback, mymodel = model_setting(train_sequence, train_label, word2vec_matrix())

Calculate F1-score to check if our model trains well

In [ ]:
#### f1-score

from sklearn.metrics import classification_report
y_pred_result = mymodel.predict(my_testing, batch_size=1024) # prediction
y_pred_result = np.argmax(y_pred_result, axis=1) # decode
results = [emotion_array[i]  for i in y_pred_result] # change from number to emotion
print(classification_report(y_true=trains['emotion'], y_pred=results))

Predict test data and save to .csv file

In [ ]:
## predict
pred_result = mymodel.predict(text_sequence, batch_size=1024)

# Returns the indices of the maximum values along an axis
pred_result = np.argmax(pred_result, axis=1) # use argmax to decode

# change from number to emotion (using emotion_array)
results = [emotion_array[i]  for i in pred_result]
pred = tests
pred['emotion'] = results
# delete other information from dataframe
pred = pred.drop('hashtags', axis=1)
pred = pred.drop('text', axis=1)
pred = pred.drop('identification', axis=1)
pred.rename(columns={'tweet_id':'id'}, inplace=True) # format
pred.head()
pred.to_csv('prediction_{}.csv'.format(datetime.now()), index=False) # save file
